# SHP2CSV algorithm
## Importing libraries

In [1]:
from shapely.geometry import shape
from shapely.ops import unary_union
import fiona
import networkx as nx
import itertools
import matplotlib.pyplot as plt
from IPython.core.debugger import Tracer
from shutil import copyfile
import pandas as pd

## Importing and reading shp file

## Converting `geoms` into a planar graph

In [2]:
file_info = fiona.open("shp/Bogota_Nodos/Bogota_Edges.shp")

## Creating a Graph object in order to print it
this part of the code is not needed, it is just for having an idea of the graph that is going to be exported

## Plotting the graph

## Creating sets

In [3]:
y = set()
x = set()
lengths = set()

nodes = set()
edges = set()
edges_prime = set()

nodes_norm = list()
edges_norm = set()

## Creating a list of nodes

In [4]:
for line in file_info:
    for node in line['geometry']['coordinates']:
        nodes.add((round(node[0],3),round(node[1],3)))
        x.add(round(node[0],3))
        y.add(round(node[1],3))

nodes = list(nodes)

## Normalizing the nodes coordinates

In [5]:
# Getting the max and min for x and y axis
x_max = max(x)
x_min = min(x)
y_max = max(y)
y_min = min(y)

In [6]:
df3 = pd.DataFrame(data=[[x_min,x_max,y_min,y_max]], columns=['x_min','x_max','y_min','y_max'])
df3.to_csv('../../distributions/boundaries.csv', index=False)

In [7]:
# Normalizing

for node in nodes:
    node_norm = (nodes.index(node), [(node[0]-x_min)/(x_max-x_min),(node[1]-y_min)/(y_max-y_min)])
    nodes_norm.append(node_norm)
    
nodes_norm = [(item[0], [round(item[1][0],3), round(item[1][1],3)]) for item in nodes_norm]

## Creating a list of edges

In [8]:
for line in file_info:
    last_coord = None
    for coord in line['geometry']['coordinates']:
        if last_coord != None:
            length = ((coord[0]-last_coord[0])**2+(coord[1]-last_coord[1])**2)**0.5
            edge = ((nodes.index((round(last_coord[0],3),round(last_coord[1],3))),nodes.index((round(coord[0],3),round(coord[1],3)))),length,line['properties']['Id'])
            edges.add(edge)
        last_coord = coord
        
    
edges = [edge for edge in edges if edge[0][1] != edge[0][0]]


## Normalizing the cost of the edges

In [9]:
length_max = max(list(zip(*edges))[1])
length_min = min(list(zip(*edges))[1])

# normalizing length
edges_norm = [(edges.index(edge),edge[0],(edge[1]-length_min)/(length_max-length_min),edge[2]) for edge in edges]

## Exporting results to csv

In [10]:
# Writing csv
import csv

# vertices
with open("vertices.csv", "w") as output:
    writer = csv.writer(output, lineterminator='\n')
    for val in nodes_norm:
        writer.writerow([val[0],val[1][0],val[1][1]])
        
# edges
with open("edges.csv", "w") as output:
    writer = csv.writer(output, lineterminator='\n')
    for val in edges_norm:
        writer.writerow([val[0], val[1][0], val[1][1], val[2], val[3]])

copyfile('edges.csv', '../../streets/edges.csv');
copyfile('vertices.csv', '../../streets/vertices.csv');

In [11]:
coords = []

for coord in list(zip(*edges))[0]:
    coord1 = (nodes[coord[0]], nodes[coord[1]])
    coords.append(coord1)

In [12]:
coords

[((-74.128, 4.65), (-74.12, 4.643)),
 ((-74.129, 4.539), (-74.121, 4.544)),
 ((-74.105, 4.595), (-74.106, 4.599)),
 ((-74.045, 4.728), (-74.047, 4.717)),
 ((-74.05, 4.728), (-74.051, 4.724)),
 ((-74.106, 4.627), (-74.093, 4.641)),
 ((-74.054, 4.643), (-74.058, 4.641)),
 ((-74.157, 4.641), (-74.166, 4.635)),
 ((-74.099, 4.608), (-74.095, 4.604)),
 ((-74.064, 4.668), (-74.064, 4.675)),
 ((-74.077, 4.62), (-74.074, 4.615)),
 ((-74.127, 4.671), (-74.131, 4.667)),
 ((-74.113, 4.689), (-74.114, 4.681)),
 ((-74.137, 4.591), (-74.125, 4.594)),
 ((-74.113, 4.667), (-74.107, 4.675)),
 ((-74.074, 4.615), (-74.078, 4.609)),
 ((-74.053, 4.684), (-74.049, 4.686)),
 ((-74.069, 4.623), (-74.071, 4.62)),
 ((-74.057, 4.688), (-74.053, 4.687)),
 ((-74.067, 4.662), (-74.064, 4.668)),
 ((-74.034, 4.695), (-74.029, 4.693)),
 ((-74.048, 4.736), (-74.049, 4.734)),
 ((-74.099, 4.547), (-74.104, 4.563)),
 ((-74.067, 4.672), (-74.064, 4.668)),
 ((-74.108, 4.697), (-74.096, 4.688)),
 ((-74.062, 4.659), (-74.065, 